In [23]:
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import scipy.signal as signal
import matplotlib.gridspec as gridspec
#import seaborn as sns
import networkx as nx
#sns.reset_defaults()

In [24]:
channel=pickle.load(open("pickles/channel.pickle","rb"))
locations=pickle.load(open("pickles/locations.pickle","rb"))

title="Stimulus S1 obj - Alcohol"
f=open("data/alcohol/co2a0000364.rd.000","r")

title="Stimulus S2 nomatch - Alcohol"
f=open("data/alcohol/co2a0000364.rd.007","r")

title="Stimulus S2 match err - Alcohol"
f=open("data/alcohol/co2a0000364.rd.011","r")

title="Stimulus S2 match - Alcohol"
f=open("data/alcohol/co2a0000364.rd.009","r")

In [36]:
# f=open("co2c1000367.rd.000","r")
# f=open("co2a0000364.rd.055","r")
# title="Stimulus S2 nomatch - Control"
# f=open("data/control/co2c0000337.rd.021","r")

# title="Stimulus S1 obj - Control"
# f=open("data/control/co2c0000337.rd.000","r")

title="Stimulus S1 obj - Alcohol"
f=open("data/alcohol/co2a0000364.rd.000","r")

data=OrderedDict()

for l in f:
    row=l.split()
    if(l[0]=="#" and len(row)==4 and row[2]=="chan"):
        if row[1] not in data:
            data[row[1]]=OrderedDict()

    elif len(row)==4 and row[0]!="#":
        #print(data,row)
        data[row[1]][int(row[2])]=float(row[3])  
        
        
        
waveData,gamma,beta,alpha,theta,delta=[],[],[],[],[],[]
wins=[41,31,65,151,191]
for sensor in channel:
    wave=[data[sensor][i] for i in data[sensor]]
    waveData.append(wave)
    sdata,gpeel=savitzkyGolay(wave,[wins[0],3])
    sdata,bpeel=savitzkyGolay(wave,[wins[1],3])
    sdata,apeel=savitzkyGolay(wave,[wins[2],3])
    sdata,tpeel=savitzkyGolay(wave,[wins[3],3])
    sdata,dpeel=savitzkyGolay(wave,[wins[4],3])
    
    waveData.append(wave)
    gamma.append(gpeel)
    beta.append(bpeel)
    alpha.append(apeel)
    theta.append(tpeel)
    delta.append(dpeel)

waveMatrix=np.corrcoef(waveData)
gammaMatrix=np.corrcoef(gamma)
betaMatrix=np.corrcoef(beta)
alphaMatrix=np.corrcoef(alpha)
thetaMatrix=np.corrcoef(theta)
deltaMatrix=np.corrcoef(delta)
   
        
plotNetworkGraph(waveMatrix,gammaMatrix,betaMatrix,alphaMatrix,thetaMatrix,deltaMatrix,locations,channel,"A_s1obj.png",[0.5,0.7,0.9],5)

A_s1obj.png
A_s1obj.png
A_s1obj.png


In [2]:
def savitzkyGolay(data, parameters):
    #print(parameters)
    smooth_data = signal.savgol_filter(data, parameters[0], parameters[1])
    peel = np.array(data[-len(smooth_data):]) - np.array(smooth_data)
    return smooth_data, peel

In [35]:
def plotNetworkGraph(waveMatrix,gammaMatrix,betaMatrix,alphaMatrix,thetaMatrix,deltaMatrix,locations,nodeNames, filename, thresholds, nodeSize):
    
    cmap=plt.get_cmap("binary")
    for thres in thresholds:
        G_wave=nx.Graph()
        G_gamma = nx.Graph()
        G_beta = nx.Graph()
        G_alpha = nx.Graph()
        G_theta = nx.Graph()
        G_delta = nx.Graph()

        Graphs = [G_wave,G_gamma,G_beta,G_alpha,G_theta,G_delta]
        for name in nodeNames:
            x=locations[name][0]
            y=locations[name][1]
            #print(latitude,longitude)
            
            G_wave.add_node(name,pos=(x,y))
           
            G_gamma.add_node(name,pos=(x,y))

            G_beta.add_node(name,pos=(x,y))
    
            G_alpha.add_node(name,pos=(x,y))
       
            G_theta.add_node(name,pos=(x,y))
          
            G_delta.add_node(name,pos=(x,y))
        for i in range(len(gammaMatrix)):
            for j in range(i+1,len(gammaMatrix)):
                    node1 = nodeNames[i]
                    node2 = nodeNames[j]
                    
                    waveC=waveMatrix[i][j]
                    gammaC = gammaMatrix[i][j]
                    betaC = betaMatrix[i][j]
                    alphaC = alphaMatrix[i][j]
                    thetaC = thetaMatrix[i][j]
                    deltaC = deltaMatrix[i][j]

                    highWave=(waveC>thres)
                    highGamma = (gammaC > thres)
                    highBeta = (betaC > thres)
                    highAlpha = (alphaC > thres)
                    highTheta = (thetaC > thres)
                    highDelta = (deltaC > thres)
                    
                    
                    if highWave:
                        G_wave.add_edge(node1, node2, weight = waveC) 
                    if highGamma:
                        G_gamma.add_edge(node1, node2, weight = gammaC) 
                    if highBeta:
                        G_beta.add_edge(node1, node2, weight = betaC) 
                    if highAlpha:
                        G_alpha.add_edge(node1, node2, weight = alphaC) 
                    if highTheta:
                        G_theta.add_edge(node1, node2, weight = thetaC) 
                    if highDelta:
                        G_delta.add_edge(node1, node2, weight = deltaC) 

        plt.close()
        plt.figure(figsize = (7,4), dpi = 300)
        gs = gridspec.GridSpec(1,4)
        
        plt.axis('off')

        plt.subplot(gs[0,0:2])
        plt.title('Wave',fontsize = 7)
        edgewidth_Amplitude = [ d['weight'] for (u,v,d) in G_wave.edges(data=True)]
        pos=nx.get_node_attributes(G_wave,'pos')
        nx.draw_networkx_nodes(G_wave,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
        nx.draw_networkx_edges(G_wave, pos, width=edgewidth_Amplitude ,cmap=cmap,edge_color =  [cmap(i) for i in edgewidth_Amplitude], alpha = [cmap(i) for i in edgewidth_Amplitude])
        # edge_color = edgewidth_Amplitude  alpha = edgewidth_Amplitude
        plt.axis('off')

        plt.subplot(gs[0,2:4])
        plt.title('Peel(41)',fontsize = 7)
        edgewidth_Amplitude = [ d['weight'] for (u,v,d) in G_gamma.edges(data=True)]
        pos=nx.get_node_attributes(G_gamma,'pos')
        nx.draw_networkx_nodes(G_gamma,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
        # 
        nx.draw_networkx_edges(G_gamma, pos,cmap=cmap,edge_color =  [cmap(i) for i in edgewidth_Amplitude], alpha = [cmap(i) for i in edgewidth_Amplitude], width=edgewidth_Amplitude)
        #Bmap_Amplitude.drawlsmask(land_color='coral',ocean_color='aqua',lakes=True)
        plt.axis('off')


#         plt.subplot(gs[1,0:2])
#         plt.title('Peel(31)',fontsize = 7)
#         edgewidth_Peel = [ d['weight'] for (u,v,d) in G_beta.edges(data=True)]	
#         color= [cmap(i) for i in edgewidth_Peel]
#         pos=nx.get_node_attributes(G_beta,'pos')
#         nx.draw_networkx_nodes(G_beta,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
#         nx.draw_networkx_edges(G_beta, pos,cmap=cmap, edge_color = color, width=edgewidth_Peel, alpha = color)
#         plt.axis('off')

#         plt.subplot(gs[1,2:4])
#         plt.title('Peel(65)',fontsize = 7)
#         edgewidth_OnlyAmplitude = [ d['weight'] for (u,v,d) in G_alpha.edges(data=True)]
#         color= [cmap(i) for i in edgewidth_OnlyAmplitude]
#         pos=nx.get_node_attributes(G_alpha,'pos')
#         nx.draw_networkx_nodes(G_alpha,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
#         nx.draw_networkx_edges(G_alpha, pos,cmap=cmap, edge_color = color, width=edgewidth_OnlyAmplitude, alpha = color)
#         plt.axis('off')


#         plt.subplot(gs[2,0:2])
#         plt.title('Peel(151)',fontsize = 7)
#         edgewidth_OnlyPeel = [ d['weight'] for (u,v,d) in G_theta.edges(data=True)]	
#         color= [cmap(i) for i in edgewidth_OnlyPeel]
#         pos=nx.get_node_attributes(G_theta,'pos')
#         nx.draw_networkx_nodes(G_theta,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
#         nx.draw_networkx_edges(G_theta, pos,cmap=cmap, edge_color = color, width=edgewidth_OnlyPeel, alpha = color)
#         plt.axis('off')

#         plt.subplot(gs[2,2:4])
#         plt.title('Peel(191)',fontsize = 7)
#         edgewidth_AmplitudenPeel = [ d['weight'] for (u,v,d) in G_delta.edges(data=True)]
#         color= [cmap(i) for i in edgewidth_AmplitudenPeel]
#         pos=nx.get_node_attributes(G_delta,'pos')
#         nx.draw_networkx_nodes(G_delta,pos, node_color = 'r', node_size = nodeSize, alpha = 0.7)
#         nx.draw_networkx_edges(G_delta, pos,cmap=cmap, edge_color = color, width=edgewidth_AmplitudenPeel, alpha = color)
#         plt.axis('off')

        plt.tight_layout()
        print(filename)
        
        #plt.suptitle(title+": Cross-Correlation: Threshold-{}".format(thres),size=8.5)
        plt.suptitle("Alcohol: Threshold-{}".format(thres),size=8.5)
        plt.subplots_adjust(top=0.88)
        #plt.show()
        plt.savefig("waveAndPeel/SUPP/{}_".format(filename)+str(thres)+".png", dpi = 300)


In [59]:
gammaMatrix

array([[ 1.        ,  0.84896132,  0.82432499, ...,  0.58088877,
         0.50274966,  0.51063296],
       [ 0.84896132,  1.        ,  0.58826387, ...,  0.46216847,
         0.2828387 ,  0.28834548],
       [ 0.82432499,  0.58826387,  1.        , ...,  0.60036895,
         0.56585355,  0.56613307],
       ..., 
       [ 0.58088877,  0.46216847,  0.60036895, ...,  1.        ,
         0.78313012,  0.79364391],
       [ 0.50274966,  0.2828387 ,  0.56585355, ...,  0.78313012,
         1.        ,  0.9921744 ],
       [ 0.51063296,  0.28834548,  0.56613307, ...,  0.79364391,
         0.9921744 ,  1.        ]])

In [60]:
deltaMatrix

array([[ 1.        ,  0.91423466,  0.87838901, ...,  0.664659  ,
         0.53994722,  0.53702875],
       [ 0.91423466,  1.        ,  0.69321198, ...,  0.5432895 ,
         0.36038614,  0.35879498],
       [ 0.87838901,  0.69321198,  1.        , ...,  0.80390229,
         0.73665314,  0.73233191],
       ..., 
       [ 0.664659  ,  0.5432895 ,  0.80390229, ...,  1.        ,
         0.87460886,  0.87435113],
       [ 0.53994722,  0.36038614,  0.73665314, ...,  0.87460886,
         1.        ,  0.9986144 ],
       [ 0.53702875,  0.35879498,  0.73233191, ...,  0.87435113,
         0.9986144 ,  1.        ]])

In [75]:
waveMatrix

array([[ 1.        ,  1.        ,  0.91942025, ...,  0.67419604,
         0.67319233,  0.67319233],
       [ 1.        ,  1.        ,  0.91942025, ...,  0.67419604,
         0.67319233,  0.67319233],
       [ 0.91942025,  0.91942025,  1.        , ...,  0.43290978,
         0.43195384,  0.43195384],
       ..., 
       [ 0.67419604,  0.67419604,  0.43290978, ...,  1.        ,
         0.99977685,  0.99977685],
       [ 0.67319233,  0.67319233,  0.43195384, ...,  0.99977685,
         1.        ,  1.        ],
       [ 0.67319233,  0.67319233,  0.43195384, ...,  0.99977685,
         1.        ,  1.        ]])